# 量子态层析

*版权所有 (c) 2022 百度量子计算研究所，保留所有权利。*

## 内容概要

在迈向大规模量子计算的道路上，一个核心挑战是提高量子态质量。那么一个能够高效精确刻画量子态的方法就显得十分重要。
量子态层析 （Quantum State Tomography, QST）是一种可以通过测量来精确刻画量子态具体表达形式的方法。
本教程将介绍量子态层析的理论及其在[百度量子平台](https://quantum.baidu.com/)上的实践。

## 理论
量子态层析的基本原理是对量子态进行多次不同基下的测量，由测量结果来近似量子态在这组基下的概率，最终推导得到量子态的具体表达形式。

### Pauli Transfer Matrix 表达
使用 Pauli Transfer Matrix (PTM) 可以很方便地表示密度矩阵之间的映射。在这种表达下，一个 $d=2^n$ 维度希尔伯特空间（Hilbert Space）中的密度矩阵 $\rho$ 可以被表示为 $d^2$ 维度希尔伯特-施密特空间（Hilbert-Schmidt space）的一个向量 $| \rho \rangle \rangle $ 。在希尔伯特-施密特空间中，内积被定义为：

$$
    \langle\langle A | B \rangle\rangle = \textrm{Tr} \left[ A^\dagger B \right],
$$

此处 $A, \; B$ 都是线性算符。
理论上我们可以自由地选取对应希尔伯特空间下的一组完备基进行转换，但为了方便，我们通常选择 Pauli 基。单量子比特情况下， Pauli 基一共有 $4$ 个：

$$
\begin{align}
    I = \left[\begin{matrix} 1 & 0 \\ 0 & 1 \end{matrix}\right], \;
    X = \left[\begin{matrix} 0 & 1 \\ 1 & 0 \end{matrix}\right], \;
    Y = \left[\begin{matrix} 0 & -i \\ i & 0 \end{matrix}\right], \;
    Z = \left[\begin{matrix} 1 & 0 \\ 0 & -1 \end{matrix}\right].
\end{align}
$$

我们将 Pauli 基进行归一化处理，即 $P_k \to P_k / \sqrt{d}$ ，这样我们就可以得到一组标准正交基。

$$
    P_k \in \left\{ I / \sqrt{2}, X / \sqrt{2},  Y / \sqrt{2},  Z / \sqrt{2} \right\}^{\otimes n}.
$$

综上，我们可以将一个量子态表达为如下向量形式：

$$
    | \rho \rangle \rangle = \sum_k |k\rangle\rangle \langle \langle k| \rho \rangle\rangle,\sum_k |k\rangle\rangle \langle \langle k| = I,
$$

其中 $|k \rangle\rangle$ 是由 $P_k$ 转换而来的向量，

$$
    \langle\langle k | \rho \rangle\rangle = \textrm{Tr} \left[ P_k \rho \right].
$$

因此，我们也可以利用向量形式来推导对应的密度矩阵形式

$$
    \rho = \sum_k \textrm{Tr} \left[P_k \rho\right] P_k.
$$


### 态层析方法
量子态层析的目标是通过测量操作获取未知量子态 $\rho$ 在 PTM 表达下 $| \rho \rangle \rangle$ 的各个元素。我们通常选择一组完备的测量算符 $\left\{ E_j \right\}_{j=1}^M (M \geq d^2)$ 来获取 $| \rho \rangle \rangle$ 的各个元素。通常情况下，一次测量结果的期望（或者说是概率），有如下表达形式：

$$
    \forall j \in [M],\; p_j := \langle\langle E_j | \rho \rangle \rangle = \textrm{Tr} \left[ E_j \rho \right].
$$

在其中插入完备的 Pauli 基，则有：

$$
    p_j = \langle\langle E_j | \rho \rangle\rangle = \langle\langle E_j |
    \left( \sum_{k=1}^{d^2} | k \rangle\rangle \langle\langle k | \right) | \rho \rangle\rangle
    = \sum_{k=1}^{d^2} \langle\langle E_j | k \rangle \rangle \langle \langle k | \rho \rangle \rangle,
$$

此处 $\langle\langle E_j | k \rangle\rangle = \textrm{Tr} \left[ P_k E_j \right]$, $\langle \langle k | \rho \rangle \rangle = \textrm{Tr} \left[ P_k \rho \right]$。
因为 $\left\{ E_j \right\}_j$ 是在实验前就选取好了的，所以这里我们可以定义一个已知的矩阵 $A$ ，它的组成元素为：

$$
    A_{jk} = \langle\langle E_j | k \rangle\rangle \Rightarrow
    A = \left[
         \begin{matrix}
           \langle\langle E_1 | 1 \rangle\rangle & \langle\langle E_1 | 2 \rangle\rangle & \dots & \langle\langle E_1 | d^2 \rangle\rangle \\
           \langle\langle E_2 | 1 \rangle\rangle & \langle\langle E_2 | 2 \rangle\rangle & \dots & \langle\langle E_2 | d^2 \rangle\rangle \\
           \vdots & \vdots & \ddots & \vdots \\
           \langle\langle E_M | 1 \rangle\rangle & \langle\langle E_M | 2 \rangle\rangle & \dots & \langle\langle E_M | d^2 \rangle\rangle \\
          \end{matrix}
      \right].
$$

我们可以制备多个相同态，将这些态在同一测量算符 $E_j$ 下进行测量操作，由测量结果的平均值来获得 $p_j$ 的估计值 $\hat{p}_j = \sum_{i=1}^N \hat{p}_{i|j} / N $ ，此处 $\hat{p}_{i|j} = 0$ 或 $1$ ，对应于第 $i$ 次测量的结果。不难发现， $\mathbb{E} (\hat{p}_j)= \sum_{i=1}^N \mathbb{E} (\hat{p}_{i|j}) / N = p_j$ 。由大数定律可知，当我们的样本数目足够多时，我们可以近似认为 $\hat{p}_j \approx p_j$ 。
然后我们定义一个 $M \times 1$ 的实数矩阵：

$$
    \hat{p} := \left[ \begin{matrix} \hat{p}_1 \\  \hat{p}_2 \\ \vdots \\ \hat{p}_M \\  \end{matrix}\right],
$$

需要注意的是这个矩阵的所有元素均是我们实验上可以测得的，如下图所示，我们将 $\rho$ 在每个 $E_j$ 下多次测量，得到对应的期望值 $\hat{p}_j$ 。
 
![QST](./figures/qst-circuit.png "图 1 ：QST 电路")

然后我们还需要注意到：

$$
    | \rho \rangle\rangle =
    \left[ \begin{matrix} \langle\langle 1 | \rho \rangle\rangle \\ \langle\langle 2 | \rho \rangle\rangle \\ \vdots \\ \langle\langle d^2 | \rho \rangle\rangle \\  \end{matrix} \right].
$$

用矩阵来表达我们最初的问题，也就是求解 $| \rho \rangle \rangle$：

$$
    \hat{p} = A | \rho \rangle \rangle,
$$

其中，$A$ 和 $\hat{p}$ 都是已知的。
我们可以选取多种方法进行处理，最直接的方法便是求逆，

$$
    | \rho \rangle\rangle = \left(A^\dagger A\right)^{-1} A^\dagger \hat{p}.
$$

如果 $A$ 矩阵是可逆的，则有

$$
    | \rho \rangle\rangle = A^{-1} \hat{p}.
$$

除此之外，我们还可以选择最大似然估计进行求解。
最后，我们回顾整个流程。
 
![QST](./figures/qst-chart.png "图 2 ：QST 流程图")
 

## 实践

现在，我们将演示如何利用 QST 来贝尔态进行分析。

首先，我们导入一些必要的包。

In [1]:
from QCompute import *
from qcompute_qep.tomography import StateTomography
from qcompute_qep.quantum.metrics import state_fidelity
from qcompute_qep.utils.circuit import circuit_to_state

然后，我们搭建制备贝尔态的量子电路。

In [2]:
qp = QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(2)
H(qp.Q[0])
CX(qp.Q[0], qp.Q[1])



![QST example](./figures/qst-Bell-example.png "图 3 ：Bell 态制备电路")

现在，我们成功搭建如上电路。
在进行量子态层析前，我们求解理想贝尔态的密度矩阵作为参考。

In [ ]:
ideal_state = circuit_to_state(qp, vector=False)
print("ideal state operator...")
print(ideal_state)

接下来我们需要选择量子计算机的类型，可以选择模拟器或量子计算机真机，这里我们选择的是百度本地量子模拟器。剩下的步骤就很简单了，只需要调用我们的 StateTomography 模块则可完成真实量子态的层析，这里我们选择对每个测量算符 $E_j$ 做 $8192$ 次测量，采用求逆的方式进行 PTM 的估计。

In [ ]:
# 请登录“量易伏”平台（https://quantum-hub.baidu.com/）获取 Token
Define.hubToken = "TOKEN"
qc = BackendName.CloudBaiduQPUQian
# Initialize a StateTomography instance
st = StateTomography()
# Call the tomography procedure and obtain the noisy quantum state
noisy_state = st.fit(qp, qc, method='inverse', shots=8192)

量子信息中，我们常用保真度（fidelity）来衡量两个量子态的接近程度，因此我们用保真度（fidelity）来评估我们 QST 的效果优劣。


In [5]:
print('Fidelity between the ideal and noisy states: F = {:.5f}'.format(st.fidelity))

Fidelity between the ideal and noisy states: F = 0.80682


## 总结

本教程旨在介绍 QST 的原理以及如何在[百度量子平台](https://quantum.baidu.com/)上实现。

## 参考资料
[1] Greenbaum, Daniel. "Introduction to quantum gate set tomography." [arXiv](https://arxiv.org/abs/1509.02921) preprint arXiv:1509.02921 (2015).
